In [1]:
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

import pandas as pd
import time
import numpy as np
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

In [44]:
house_df = pd.read_csv('data/house_mean_imputation.csv')
#house_df = pd.read_csv('data/ready_to_use_dataset.csv')
house_df = house_df.drop_duplicates(['year', 'state', 'district', 'name'])
display(house_df.shape)

(9802, 20)

In [45]:
house_df['last_house_majority'].unique()

array(["(9611, 'R')", 'D', 'R', "(9729, 'R')", "(9365, 'R')",
       "(9336, 'R')", "(9206, 'R')", "(9755, 'R')", "(9715, 'R')",
       "(9888, 'R')", "(9283, 'R')", "(9475, 'R')", "(9412, 'R')",
       "(9696, 'R')", "(9209, 'R')", "(9303, 'R')", "(9760, 'R')",
       "(9819, 'R')", "(9203, 'R')", "(9848, 'R')", "(9860, 'R')",
       "(9745, 'R')", "(9463, 'R')", "(9210, 'R')", "(9769, 'R')",
       "(9744, 'R')", "(9558, 'R')"], dtype=object)

In [37]:
#house_df2=house_df.set_index(['year', 'state', 'district', 'name']).dropna().copy()
house_df2=house_df.dropna().copy()
house_df2_districts=house_df2[['state','district']]
house_df2=house_df2.drop('state', axis=1).drop('district', axis=1).drop('name', axis=1)
house_df2['party']=house_df2['party'].replace(['D', 'R'], [0, 1])
house_df2['president_party']=house_df2['president_party'].replace(['D', 'R'], [0, 1])
house_df2['last_house_majority']=house_df2['last_house_majority'].replace(['D', 'R'], [0, 1])
house_df2=house_df2.drop('first_time_elected', axis=1)
data_train, data_test=house_df2[house_df2['year']!=2018], house_df2[house_df2['year']==2018]

x_train, y_train=data_train.drop('won', axis=1), data_train['won']

x_test, y_test=data_test.drop('won', axis=1), data_test['won']
baselineLogRegr=LogisticRegressionCV(cv=5, penalty='l2').fit(x_train, y_train)

In [38]:
display(house_df[(house_df['year']!=2018)&(house_df['state']=='Alabama')&(house_df['district']=='District 1')])

,district,is_incumbent,name,party,percent,state,votes,won,year,first_time_elected,count_victories,unemployement_rate,is_presidential_year,president_can_be_re_elected,president_party,president_overall_avg_job_approval,last_D_house_seats,last_R_house_seats,last_house_majority,fundraising
4254,District 1,0.0,Judy Belk,D,38.0,Alabama,67507.0,0,2002,NaN,0,5.7,0.0,1.0,R,0.494,212.0,221.0,R,NaN
4255,District 1,0.0,Jo Bonner,R,61.0,Alabama,108102.0,1,2002,2002.0,0,5.7,0.0,1.0,R,0.494,212.0,221.0,R,NaN
5021,District 1,NaN,Judy Belk,D,37.0,Alabama,93938.0,0,2004,NaN,0,5.5,1.0,0.0,R,0.494,205.0,229.0,R,NaN
5022,District 1,1.0,Jo Bonner,R,63.0,Alabama,161067.0,1,2004,2002.0,1,5.5,1.0,0.0,R,0.494,205.0,229.0,R,NaN
5556,District 1,1.0,Jo Bonner,R,68.0,Alabama,112944.0,1,2006,2002.0,2,4.4,0.0,0.0,R,0.494,202.0,232.0,R,NaN
5576,District 1,0.0,Vivian Beckerle,D,32.0,Alabama,52770.0,0,2006,NaN,0,4.4,0.0,0.0,R,0.494,202.0,232.0,R,NaN
5742,District 1,1.0,Jo Bonner,R,98.0,Alabama,210660.0,1,2008,2002.0,3,6.5,1.0,1.0,D,0.480,233.0,202.0,D,NaN
6316,District 1,1.0,Jo Bonner,R,83.0,Alabama,129063.0,1,2010,2002.0,4,9.4,0.0,1.0,D,0.480,257.0,178.0,D,900198.76
7326,District 1,1.0,Jo Bonner,R,97.9,Alabama,196374.0,1,2012,2002.0,5,10.9,1.0,0.0,D,0.480,193.0,242.0,R,1177293.00
8227,District 1,0.0,Burton LeFlore,D,31.7,Alabama,48278.0,0,2014,NaN,0,8.3,0.0,0.0,D,0.480,201.0,234.0,R,4001.05


In [42]:
#Accuracy is defined as (TP+TN)/n
def printAccuracy(y_train, y_pred_train, y_test, y_pred_test):
    print('Training Set Accuracy: \t{:.2%}'.format(np.sum(y_train == y_pred_train) / len(y_train)))
    print('Test Set Accuracy: \t{:.2%}'.format(np.sum(y_test == y_pred_test) / len(y_test)))

y_pred_train=baselineLogRegr.predict(x_train)
y_pred_test=baselineLogRegr.predict(x_test)
printAccuracy(y_train, y_pred_train, y_test, y_pred_test)
print('Amount of districts in the predictions: {:.1%} of the total'.format(len(x_test.join(house_df2_districts).groupby(['state', 'district']).count())/435))

Training Set Accuracy: 	96.24%
Test Set Accuracy: 	94.12%
Amount of districts in the predictions: 67.1% of the total


In [43]:
#Add index to predictions from X set
y_pred_train=pd.DataFrame(y_pred_train, index=x_train.index, columns=['y_pred_train'])
y_pred_test=pd.DataFrame(y_pred_test, index=x_test.index, columns=['y_pred_test'])

#Join districts data by index
house_df2_pred_districts=house_df2_districts.join(x_train)
house_df2_pred_districts.groupby(['state', 'district']).count()

is_incumbent  party  percent  votes  year  \
state         district                                                 
Alabama       District 1              4      4        4      4     4   
              District 2              4      4        4      4     4   
              District 3              4      4        4      4     4   
              District 4              4      4        4      4     4   
              District 5              4      4        4      4     4   
              District 6              4      4        4      4     4   
              District 7              4      4        4      4     4   
Arizona       District 1              4      4        4      4     4   
              District 2              6      6        6      6     6   
              District 3              4      4        4      4     4   
              District 4              1      1        1      1     1   
              District 5              3      3        3      3     3   
              District 6              4      4        4      4     4   
              District 7              4      4        4      4     4   
              District 8              1      1        1      1     1   
              District 9              3      3        3      3     3   
Arkansas      District 3              4      4        4      4     4   
              District 4              2      2        2      2     2   
California    District 1              4      4        4      4     4   
              District 10             4      4        4      4     4   
              District 11             4      4        4      4     4   
              District 12             4      4        4      4     4   
              District 13             3      3        3      3     3   
              District 14             4      4        4      4     4   
              District 15             3      3        3      3     3   
              District 16             4      4        4      4     4   
              District 17             3      3        3      3     3   
              District 18             4      4        4      4     4   
              District 19             4      4        4      4     4   
              District 2              4      4        4      4     4   
...                                 ...    ...      ...    ...   ...   
Utah          District 4              2      2        2      2     2   
Virginia      District 1              2      2        2      2     2   
              District 10             4      4        4      4     4   
              District 11             0      0        0      0     0   
              District 2              2      2        2      2     2   
              District 3              2      2        2      2     2   
              District 4              3      3        3      3     3   
              District 5              3      3        3      3     3   
              District 7              3      3        3      3     3   
              District 8              0      0        0      0     0   
              District 9              3      3        3      3     3   
Washington    District 1              3      3        3      3     3   
              District 2              4      4        4      4     4   
              District 3              3      3        3      3     3   
              District 4              3      3        3      3     3   
              District 5              3      3        3      3     3   
              District 6              3      3        3      3     3   
              District 7              3      3        3      3     3   
              District 8              3      3        3      3     3   
              District 9              3      3        3      3     3   
West Virginia District 1              3      3        3      3     3   
              District 2              3      3        3      3     3   
              District 3              2      2        2      2     